In [ ]:
!pip install transformers datasets
!pip install rouge-score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=4c44faf24b2f063680c1563d

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import json

# JSON dosyasını yükleme
file_path = '/content/drive/MyDrive/metin_ozetleme_2/train.json'  # JSON dosyanızın yolu
with open(file_path, 'r', encoding='utf-8') as f:
    train = json.load(f)



In [ ]:
import pandas as pd

# JSON dosyalarını yükleme
train_data = pd.read_json('/content/drive/MyDrive/metin_ozetleme_2/train.json' )
val_data = pd.read_json('/content/drive/MyDrive/metin_ozetleme_2/test.json' )
test_data = pd.read_json('/content/drive/MyDrive/metin_ozetleme_2/val.json' )

# Verilerin ilk birkaç satırını inceleyin
print("Train Data:")
print(train_data.head())
print("\nValidation Data:")
print(val_data.head())
print("\nTest Data:")
print(test_data.head())


Train Data:
   id                                               text  \
0   0  VERASET İLAMI\nTALEP EDEN : talip1\nVEKİLİ : A...   
1   1  HAKİM : hakime1\nKATİP : katip1\nTALEP EDEN :t...   
2   2  HAKİM : hakim1\nKATİP : katip1\nŞİKAYETÇİ : ki...   
3   3  DURUŞMA TARİHİ : 26/06/2024\nCELSE NO : 2.\nHA...   
4   4  DURUŞMA TARİHİ : 25/04/2024\nCELSE NO : 1.\nHA...   

                                             summary  
0  Talep eden Av. 1, İstanbul 26. İcra Dairesi'nd...  
1  Mahkeme, davacının talebi üzerine, miras bırak...  
2  Suçlama: Sanık, "Çekle ilgili karşılıksızdır i...  
3  Sanık Hakkında: 14/12/2023 tarihli 385.000 TL ...  
4  Sanığa yazılan talimat henüz dönmediği için du...  

Validation Data:
   id                                               text  \
0   0  SAYI :CELSE NO : 1\nCELSE TARİHİ : 03/05/2019\...   
1   1  DAVACI :davacı1\nVEKİLİ :Av. 1-Av. 2\nDAVALI :...   
2   2  CELSE NO : 6\nCELSE TARİHİ : 22/01/2021\nHAKİM...   
3   3  HAKİM : hakim1\nKATİP : katip1\n

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer

# Model ve tokenizer
model_name = "facebook/mbart-large-50-many-to-many-mmt"
model = MBartForConditionalGeneration.from_pretrained(model_name)
tokenizer = MBart50Tokenizer.from_pretrained(model_name, src_lang="tr_TR", tgt_lang="tr_TR")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# JSON formatındaki veri setini yükleme
data_files = {
    'train': '/content/drive/MyDrive/metin_ozetleme_2/train.json',
    'test': '/content/drive/MyDrive/metin_ozetleme_2/test.json',
    'val': '/content/drive/MyDrive/metin_ozetleme_2/val.json'
}
datasets = load_dataset('json', data_files=data_files)

# Önişleme: Metinleri tokenize etme
def preprocess_data(batch):
    inputs = tokenizer(batch['text'], max_length=1024, truncation=True, padding='max_length', return_tensors="pt")
    outputs = tokenizer(batch['summary'], max_length=256, truncation=True, padding='max_length', return_tensors="pt")
    batch['input_ids'] = inputs['input_ids']
    batch['attention_mask'] = inputs['attention_mask']
    batch['labels'] = outputs['input_ids']
    return batch

train_dataset = datasets['train'].map(preprocess_data, batched=True)
val_dataset = datasets['val'].map(preprocess_data, batched=True)
test_dataset = datasets['test'].map(preprocess_data, batched=True)

# İlk birkaç örneği inceleyin
print("Train Dataset Example:")
print(train_dataset[0])


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Train Dataset Example:
{'id': '0', 'text': 'VERASET İLAMI\nTALEP EDEN : talip1\nVEKİLİ : Av. 1\nMURİS : kişi1 \nKONU : Mirasçılık belgesi talebimizden ibarettir.\nAÇIKLAMALAR :\nİstanbul 26.İcra Dairesi Esas sayılı dosyamız borçlularından TC Kimlik numaralı, kişi1 vefat ettiği öğrenilmiştir. Muris hakkında takibe devam edilebilmesi, mirasçılarının dosyaya dahil edilebilmesi için icra dairesince tarafımıza İİK 94.maddesi gereği yetki belgesi verilmiştir.\nEkte sunulan yetki belgesi gereği, kimlik bilgileri bulunan kişi1 adına reddi miras, mirasçılık belgesinin iptali gibi davalar açılıp açılmadığının mahkemenizce araştırılmasını nihayetinde tarafımıza mirasçılık belgesinin verilmesini bilvekale arz ve talep ederiz. \nHUKUKİ NEDENLER : HMK, İİKve ilgili sair mevzuat.\nHUKUKİ DELİLER : Nüfus kayıtları, İstanbul 26.İcra Dairesi Esas sayılı dosya ve ilgili sair deliller.\nSONUÇ ve İSTEM : Yukarıda açıklanan ve mahkemece resen tespit edilecek\nhususlar birlikte gözetilerek;\n-Yukarıda ayrınt

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)




/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-8-32477dea455b>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,5.531500,3.138608
2,3.076900,2.094826
3,0.995200,1.949874
4,0.646500,1.985398
5,0.464200,2.144782
6,0.254000,2.153411
7,0.206900,2.201519
8,0.117800,2.291697
9,0.082100,2.346847
10,0.059000,2.348358


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=380, training_loss=0.5350843113896094, metrics={'train_runtime': 417.55, 'train_samples_per_second': 3.592, 'train_steps_per_second': 0.91, 'total_flos': 3250694651904000.0, 'train_loss': 0.5350843113896094, 'epoch': 20.0})

In [ ]:
import torch

# GPU kontrolü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modeli ve tokenizer'ı GPU'ya taşı
model = model.to(device)

# Tokenize edilmiş girdiyi GPU'ya taşı
def summarize_with_model(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=4096)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Tüm tensorları GPU'ya taşı
    outputs = model.generate(**inputs, max_length=1024, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test
test_text = val_dataset[1]['text']
predicted_summary = summarize_with_model(test_text)
print("Modelin Özetlediği:")
print(predicted_summary)


Modelin Özetlediği:
Mahkeme, taraflar arasındaki davada dosyanın bilirkişilere tevdi edilmesine karar vermiştir. Bilirkişi, mahkemeye gelmeden usulüne uygun yemini yaptırmıştır. Bilirkişi, bilirkişinin raporunu 60 gün içinde taraf adedinden bir fazla suret halinde ibraz etmesine karar vermiştir. Rapor ibraz edildiğinde ücretin kendisine ödenmesine karar verilmiştir. Taraflarca raporun duruşma günü beklenmeden tebliğ edilecektir. Davalı vekilleri: Av. 1, Davacı: Av. 2, Av. 3 Duruşma Tarihi: 05/11/2020
